Заполните данные о необходимых переменных:

In [49]:
RUB_USD = 0.010309087
EUR_USD = 1.052536139

Today = '29.02.2024'
Mounth = 'Feb'

Group = 'SUEK'
# Group = 'EUROCHEM'

In [50]:
from datetime import date
Output_file = "_".join([str(date.today()), Group, 'SAP', 'Debt', f'{Mounth}.xlsx'])
Sheet_in_output_file = 'Debt'
Output_file

'2024-03-21_EUROCHEM_SAP_Debt_Feb.xlsx'

Открываю следующие файлы:

In [1]:
import oracledb
import pandas as pd

oracledb.init_oracle_client('C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Oracle\\instantclient_21_13')

connection = oracledb.connect(user="RISKCUSTOM", password="xxRiscRccess174!", host="exatest2-scan.moscow.eurochem.ru", port=1521, service_name='riskdev.moscow.eurochem.ru', disable_oob= True)

In [6]:
query = """SELECT * FROM "RISKACCESS"."sapPositionArrears" WHERE "reportDate" = TO_DATE('29/02/24', 'DD/MM/YY') AND "productType" in (130,131,132,133)"""
data_export = pd.read_sql(query, con=connection)
data_export.head(2)

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_8304\3317839245.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


,##batch_id,##deleted,##inserted,##origin_name,companyCode,companyName,valuationAreaId,valuationAreaName,valuationClassId,valuationClassName,...,reportDate_month,securedContractDate_year,securedContractDate_quarter,securedContractDate_month,termEnd_year,termEnd_quarter,termEnd_month,termStart_year,termStart_quarter,termStart_month
0,34099078@WHPROD,N,2024-02-29 07:13:33,SAP,E100,"АО ""Ковдорский ГОК""",001,Оперативная область оценки,0102,Финансовая кредиторская задолженность (долгоср...,...,2,NaN,NaN,NaN,2026,2,4,2021,2,4
1,34099078@WHPROD,N,2024-02-29 07:13:33,SAP,E100,"АО ""Ковдорский ГОК""",001,Оперативная область оценки,0102,Финансовая кредиторская задолженность (долгоср...,...,2,NaN,NaN,NaN,2026,2,5,2021,2,5


In [57]:
data_export.columns[data_export.columns.str.contains('No')]

Index(['promissoryNotePayee1', 'promissoryNotePayeeName1',
       'promissoryNoteCurrentPayee', 'promissoryNoteCurrentPayeeName',
       'promissoryNoteSlip', 'stockNominalValue'],
      dtype='object')

In [5]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\OCP")
from Defs import merge_SalesUnits
from Defs import merge_Mapping
from Defs import Period
from Defs import new_list

In [104]:
data_export.loc[data_export.companyName == 'SUEK AG (Swiss)', ['businessPartnerName', 'companyName', 'productTypeName', 'purchaseValuePositionCurrency', 'purchaseValueDisplayCurrency'\
             , 'positionActivity']]

,businessPartnerName,companyName,productTypeName,purchaseValuePositionCurrency,purchaseValueDisplayCurrency,positionActivity
215,BANQUE CANTONALE VAUDOISE,SUEK AG (Swiss),Транш по овердрафту,6.616670e+03,-6.078682e+05,Пассивная позиция
216,SUEK Assets Holding AG,SUEK AG (Swiss),Транш по кредитам,1.000000e+08,-9.186920e+09,Пассивная позиция
415,Black Sand Commodities FZ-LLC,SUEK AG (Swiss),Транш по займам,3.272878e+07,3.006766e+09,Активная позиция
416,Black Sand Commodities FZ-LLC,SUEK AG (Swiss),Транш по займам,3.456862e+07,3.175792e+09,Активная позиция
417,Black Sand Commodities FZ-LLC,SUEK AG (Swiss),Транш по займам,2.806427e+07,2.578242e+09,Активная позиция
418,Black Sand Commodities FZ-LLC,SUEK AG (Swiss),Транш по займам,5.200000e+07,4.777198e+09,Активная позиция
419,"АО ""СУЭК""",SUEK AG (Swiss),Транш по займам,1.654112e+10,1.654112e+10,Активная позиция
479,SUEK Assets Holding AG,SUEK AG (Swiss),Транш по кредитам,5.000000e+06,-4.593460e+08,Пассивная позиция
480,SUEK Singapore Trading Pte. Ltd.,SUEK AG (Swiss),Транш по займам,1.952000e+05,1.793287e+07,Активная позиция
754,Black Sand Commodities FZ-LLC,SUEK AG (Swiss),Транш по займам,1.600000e+07,1.469907e+09,Активная позиция


In [99]:
data_export.columns[data_export.columns.str.contains('position')]

Index(['positionCurrency', 'positionActivity', 'position1', 'position2'], dtype='object')

In [111]:
import numpy as np

data_Debt = data_export.loc[data_export['productTypeName'].isin(['Транш по займам', 'Транш по кредитам'])].reset_index(drop=True)

data_Debt = data_Debt.groupby(['companyName', 'relationshipPartner',
       'businessPartnerName', 'creditConditionCurrency', 'termEnd']).agg({'purchaseValueDisplayCurrency' : sum,
                                                                          'purchaseValuePositionCurrency' : sum}).reset_index()

data_Debt['in_USD'] = data_Debt['purchaseValuePositionCurrency'] * np.sign(data_Debt['purchaseValueDisplayCurrency'])
data_Debt.loc[data_Debt['creditConditionCurrency'] == 'RUB', 'in_USD'] = data_Debt.loc[data_Debt['creditConditionCurrency'] == 'RUB', 'in_USD'] * RUB_USD
data_Debt.loc[data_Debt['creditConditionCurrency'] == 'EUR', 'in_USD'] = data_Debt.loc[data_Debt['creditConditionCurrency'] == 'EUR', 'in_USD'] * EUR_USD
data_Debt['purchaseValuePositionCurrency'] = data_Debt['purchaseValuePositionCurrency'] * np.sign(data_Debt['purchaseValueDisplayCurrency'])

data_Debt = data_Debt.rename(columns={"businessPartnerName": "Counterparty", "companyName": "Entity", "creditConditionCurrency": "Currency", "purchaseValuePositionCurrency": "amoutn_outstanding", "in_USD": "amount_USD_eq"}).reset_index(drop=True)

data_Debt['Entity_code'] = merge_Mapping(data_Debt, col='Entity')
data_Debt['Entity_group'] = merge_SalesUnits(data_Debt, col='Entity_code', merge_col='ocpSegment')
data_Debt['Counterparty_code'] = merge_Mapping(data_Debt, col='Counterparty')
data_Debt['Counterparty_Group'] = merge_SalesUnits(data_Debt, col='Counterparty_code', merge_col='ocpSegment')

data_Debt = Period(data_Debt, day_for_count=Today, col_with_date='termEnd')

data_Debt['Holding'] = merge_SalesUnits(data_Debt, col='Entity_code', merge_col='holding')
data_Debt = data_Debt.query('Holding == @Group').reset_index(drop=True)

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_8304\397998865.py:6: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  'businessPartnerName', 'creditConditionCurrency', 'termEnd']).agg({'purchaseValueDisplayCurrency' : sum,
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_8304\397998865.py:6: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  'businessPartnerName', 'creditConditionCurrency', 'termEnd']).agg({'purchaseValueDisplayCurrency' : sum,
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:41: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True`

In [112]:

manual1 = data_Debt.loc[data_Debt.Entity_code == 'External' ,['Entity', 'Entity_code']].drop_duplicates()
manual2 = data_Debt.loc[data_Debt.Counterparty_code == 'External' ,['Counterparty', 'Counterparty_Group']].drop_duplicates()
manual2

,Counterparty,Counterparty_Group
5,"АО ""Альфа-Банк""",External
35,"АО ""ВЭБ.ДВ""",External
39,EuroChem Finance Designated Activity Company,External
50,Банк ВТБ (ПАО),External
53,"ООО ""Биохим технологии""",External
70,"ООО ""Кинетик Капитал""",External
71,"ООО ""МИК ИНВЕСТ""",External
89,"ПАО Банк ""ФК Открытие""",External
90,ПАО Сбербанк,External
92,"ПУБЛИЧНОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""БАНК ""САНКТ-ПЕ...",External


Записываю данные в output file:

In [113]:
new_columns = ['dealClass1','dealClass2','instrumentOwner','dealSet','facility','Source']
data_Debt = data_Debt.reindex(columns=(data_Debt.columns.tolist() + new_columns))
data_Debt['Source'] = 'SAP'

data = data_Debt[['Entity', 'Entity_group', 'Counterparty', 'Counterparty_Group', 'amoutn_outstanding', 'Currency', 'amount_USD_eq', 'termEnd', 'Days', 'Period', 'dealClass1','dealClass2','instrumentOwner','dealSet','facility','Source']]

In [63]:
data.head(2)

,Entity,Entity_group,Counterparty,Counterparty_Group,amoutn_outstanding,Currency,amount_USD_eq,termEnd,Days,Period,dealClass1,dealClass2,instrumentOwner,dealSet,facility,Source
0,Eurochem Group AG (former Eurochem Trading GmbH),TRADING,EUROCHEM TERMINAL SILLAMÄ,EURSALES,-460.29,EUR,-4.844719e+02,2024-12-31,306.0,6M-1Y,NaN,NaN,NaN,NaN,NaN,SAP
1,Eurochem Group AG (former Eurochem Trading GmbH),TRADING,EUROCHEM TERMINAL SILLAMÄ,EURSALES,-43000000.00,EUR,-4.525905e+07,2026-09-20,934.0,2Y+,NaN,NaN,NaN,NaN,NaN,SAP


In [55]:
data.to_excel(Output_file, sheet_name = Sheet_in_output_file, index = False)

In [110]:
quantum_data_export = pd.read_excel('C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\OCP\\Quantum\\2024-03-21_EuroChem_quantum_Debt_Feb.xlsx')
quantum_data_export.head(2)

,entity,Entity_group,counterparty,Counterparty_Group,amoutn outstanding,Currency,amount USD eq,termEnd,Days,Period,dealClass1,dealClass2,instrumentOwner,dealSet,facility,Source
0,BRHRG,SAMSALES,Morgan Stanley BR,External,-8011.33,BRL,-1616.101732,2031-12-31,2862,2Y+,Loan,MM,EC_BORROWINGS,-,HRG Judicial recovery BRL,Quantum
1,BRHRG,SAMSALES,Morgan Stanley BR,External,-9958.01,BRL,-2008.799688,2032-12-31,3228,2Y+,Loan,MM,EC_BORROWINGS,-,HRG Judicial recovery BRL,Quantum


In [118]:
quantum_data_export['E_Cgroup'] = quantum_data_export.entity + quantum_data_export.Counterparty_Group + quantum_data_export.Currency + quantum_data_export.termEnd.astype(str) + quantum_data_export['amoutn outstanding'].abs().astype(str)
data_Debt['E_Cgroup'] = data_Debt.Entity_code + data_Debt.Counterparty_Group + data_Debt.Currency + data_Debt.termEnd.astype(str) + data_Debt.amoutn_outstanding.abs().astype(str)

iner_list = list(set(data_Debt['E_Cgroup']).intersection(set(quantum_data_export.E_Cgroup)))

iner_list

SAP_iner = data_Debt[data_Debt.E_Cgroup.isin(iner_list)]
Quantum_iner = quantum_data_export[quantum_data_export.E_Cgroup.isin(iner_list)]



In [121]:
iner_list_2 = SAP_iner.index.tolist()
iner_list_2
SAP_iner = data_Debt[data_Debt.index.isin(iner_list_2)]
data_Debt_not_iner = data_Debt[~data_Debt.index.isin(iner_list_2)]
SAP_iner.shape

(14, 22)

In [122]:
Quantum_iner.shape

(14, 17)

In [92]:
SAP_iner = SAP_iner[['Entity', 'Entity_group', 'Counterparty', 'Counterparty_Group', 'amoutn_outstanding', 'Currency', 'amount_USD_eq', 'termEnd', 'Days', 'Period', 'dealClass1','dealClass2','instrumentOwner','dealSet','facility','Source']]
data_Debt_not_iner = data_Debt_not_iner[['Entity', 'Entity_group', 'Counterparty', 'Counterparty_Group', 'amoutn_outstanding', 'Currency', 'amount_USD_eq', 'termEnd', 'Days', 'Period', 'dealClass1','dealClass2','instrumentOwner','dealSet','facility','Source']]


In [94]:
data_Debt_not_iner.to_excel(Output_file, sheet_name=Sheet_in_output_file, index=False)
new_list(SAP_iner, Output_file, 'Inersection')
new_list(quantum_data_export, Output_file, 'Debt_Quantum')

In [105]:
SAP_iner

,Entity,Entity_group,Counterparty,Counterparty_Group,amoutn_outstanding,Currency,amount_USD_eq,termEnd,Days,Period,dealClass1,dealClass2,instrumentOwner,dealSet,facility,Source
15,"АО ""Невинномысский Азот""",RUFERT,"АО ""МХК ""ЕВРОХИМ""",RUFERT,-7.175728e+09,RUB,-7.397520e+07,2024-12-19,294.0,6M-1Y,NaN,NaN,NaN,NaN,NaN,SAP
46,"ООО ""Русбонд-Еврохим""",RUFERT,"АО ""МХК ""ЕВРОХИМ""",RUFERT,4.670000e+06,USD,4.670000e+06,2024-03-13,13.0,<1M,NaN,NaN,NaN,NaN,NaN,SAP
63,"АО ""МХК ""ЕвроХим""",RUFERT,"ООО ""ЕВРОХИМ СЕВЕРНЫЙ КАВ",RUFERT,-3.266500e+08,RUB,-3.367463e+06,2024-11-29,274.0,6M-1Y,NaN,NaN,NaN,NaN,NaN,SAP
75,"АО ""МХК ""ЕвроХим""",RUFERT,"ООО ""ЕвроХим-УКК""",RUFERT,1.193359e+10,RUB,1.230244e+08,2024-12-31,306.0,6M-1Y,NaN,NaN,NaN,NaN,NaN,SAP
97,"ТОО ""ЕВРОХИМ - УДОБРЕНИЯ""",RUFERT,"ТОО ""ЕвроХим-Каратау""",RUFERT,-5.573584e+09,KZT,-5.573584e+09,2024-12-31,306.0,6M-1Y,NaN,NaN,NaN,NaN,NaN,SAP


In [106]:
Quantum_iner

,entity,Entity_group,counterparty,Counterparty_Group,amoutn outstanding,Currency,amount USD eq,termEnd,Days,Period,dealClass1,dealClass2,instrumentOwner,dealSet,facility,Source,E_Cgroup
311,RUNEV,RUFERT,MCC,RUFERT,7.175728e+09,RUB,8.036547e+07,2024-12-19,294,6M-1Y,Loan,MM,EC_LOANS,-,RUNEV-RUEMO,Quantum,RUNEVRUFERTRUB2024-12-19
1092,RUSBO,RUFERT,RUEMO,RUFERT,4.770000e+06,USD,4.770000e+06,2024-03-13,13,<1M,Loan,SEC,EC_TRADED_BONDS,MCC RuBond 09 $25m 2023-05,-,Quantum,RUSBORUFERTUSD2024-03-13
1106,RUEMO,RUFERT,EC North Caucasus,RUFERT,3.266500e+08,RUB,3.658358e+06,2024-11-29,274,6M-1Y,Loan,MM,EC_LOANS,-,RUEMO-RUNCA,Quantum,RUEMORUFERTRUB2024-11-29
1218,KZUDB,RUFERT,EuroChem-Karatau,RUFERT,5.573584e+09,KZT,1.238628e+07,2024-12-31,306,6M-1Y,Loan,MM,EC_LOANS,-,KZUDB-KZKRT,Quantum,KZUDBRUFERTKZT2024-12-31
1219,KZUDB,RUFERT,Sary-Tas,RUFERT,1.460880e+08,KZT,3.246542e+05,2024-12-31,306,6M-1Y,Loan,MM,EC_LOANS,-,KZUDB-KZSTS,Quantum,KZUDBRUFERTKZT2024-12-31
1221,KZUDB,RUFERT,Ushger,RUFERT,8.250000e+08,KZT,1.833413e+06,2024-12-31,306,6M-1Y,Loan,MM,EC_LOANS,-,KZUDB-KZUSH,Quantum,KZUDBRUFERTKZT2024-12-31
1235,RUEMO,RUFERT,UKK,RUFERT,-1.193359e+10,RUB,-1.336517e+08,2024-12-31,306,6M-1Y,Loan,MM,EC_LOANS,-,RUEMO-RUKUS,Quantum,RUEMORUFERTRUB2024-12-31


In [108]:
quantum_data_export[quantum_data_export.counterparty == 'Sary-Tas']

,entity,Entity_group,counterparty,Counterparty_Group,amoutn outstanding,Currency,amount USD eq,termEnd,Days,Period,dealClass1,dealClass2,instrumentOwner,dealSet,facility,Source,E_Cgroup
1215,KZUDB,RUFERT,Sary-Tas,RUFERT,-15000000.0,KZT,-33334.789285,2024-02-20,-9,<1M,Loan,MM,EC_LOANS,-,KZUDB-KZSTS,Quantum,KZUDBRUFERTKZT2024-02-20
1219,KZUDB,RUFERT,Sary-Tas,RUFERT,146088000.0,KZT,324654.179801,2024-12-31,306,6M-1Y,Loan,MM,EC_LOANS,-,KZUDB-KZSTS,Quantum,KZUDBRUFERTKZT2024-12-31


In [109]:
data_Debt[data_Debt.Entity =='ТОО "Сары-Тас-Удобрения"']

,Counterparty,relationshipPartner,Entity,Currency,termEnd,purchaseValueDisplayCurrency,amoutn_outstanding,amount_USD_eq,Entity_code,Entity_group,Counterparty_code,Counterparty_Group,Days,Period,Holding,dealClass1,dealClass2,instrumentOwner,dealSet,facility,Source,E_Cgroup
100,"ТОО ""ЕвроХим-Удобрения""",Прочие,"ТОО ""Сары-Тас-Удобрения""",KZT,2025-01-31,3069885.0,15000000.0,15000000.0,KZSTS,RUFERT,External,External,337.0,6M-1Y,EUROCHEM,NaN,NaN,NaN,NaN,NaN,SAP,KZSTSExternalKZT2025-01-31
